In [1]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("241205_03_reduce")
spark = SparkContext(conf=conf).getOrCreate()

24/12/05 10:49:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Reduce : 여러개의 값을 하나로 축약

In [2]:
sample_rdd = spark.parallelize([1,2,3,4,5])
sample_rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [3]:
from operator import add

sample_rdd.reduce(add)

15

In [4]:
## 만약 파티션이 나뉘어져 있다면
'''
lambda x,y : (x*2) + y
1: x=1, y=2 >> 1*2 + 2 = 4


'''

'\nlambda x,y : (x*2) + y\n1: x=1, y=2 >> 1*2 + 2 = 4\n\n\n'

In [22]:
sample_rdd1 = spark.parallelize([1,2,3,4])
sample_rdd1.reduce(lambda x,y : (x*2)+y)

26

In [24]:
sample_rdd1.glom().collect()

[[1, 2, 3, 4]]

In [27]:
# 파티션을 쪼개보기(2개)
sample_rdd2 = spark.parallelize([1,2,3,4],2) #파티션을 두개로 직접 쪼개기
sample_rdd2.glom().collect()

[[1, 2], [3, 4]]

In [28]:
sample_rdd2.reduce(lambda x,y : (x*2)+y)

18

In [35]:
# 파티션을 쪼개보기(3개)
sample_rdd3 = spark.parallelize([1,2,3,4],3) #파티션을 세개로 직접 쪼개기
sample_rdd3.reduce(lambda x,y : (x*2)+y)

18

In [38]:
## fold연산

rdd4 = spark.parallelize([2,3,4],4)
rdd4.reduce(lambda x,y : x*y)


24

In [39]:
rdd4.fold(1, lambda x,y:x*y)

24

In [43]:
# 빈 RDD
rdd5 = spark.parallelize([])

try : rdd5.reduce(lambda a,b : a+b) #RDD가 비어있어 reduce할 수 없음, 이를 해결하기 위해 예외처리
except ValueError as e : print(f"Reduce Error : {e}")

Reduce Error : Can not reduce() empty RDD


In [45]:
rdd5.fold(0, lambda a,b : a+b)

0

## reduce와 fold는 비슷한 기능을 하지만 
1. reduce()는 단순한 RDD 축약연산, 비어있지 않은 RDD의 경우
2. fold()는 일반적이고 오류가 없는 연산, 초기값을 적용해서 비어있는 RDD, 파티션의 경우에도 적용할 수 있음.

In [46]:
rdd6 = spark.parallelize([1,"a",2]) #다른 타입의 데이터를 합쳐놓음
rdd6.collect()

[1, 'a', 2]

In [53]:
try : rdd6.reduce(lambda x,y:str(x)+str(y)) 
except TypeError as e : print(f"Reduce Error : {e}")

In [51]:
rdd6.fold("",lambda x,y:str(x) + str(y))

'1a2'

In [54]:
## groupBy(), aggregate()

rdd7 = spark.parallelize([1,2,1,1,3,5,5,8])
rdd7.collect()

[1, 2, 1, 1, 3, 5, 5, 8]

In [58]:
result = rdd7.groupBy(lambda x: x%2).collect()

In [60]:
sorted([(x,sorted(y)) for (x,y) in result])

[(0, [2, 8]), (1, [1, 1, 1, 3, 5, 5])]

In [61]:
func1 = lambda x,y : (x[0]+y, x[1]+1)
func2 = lambda x,y : (x[0]+y[0], x[1]+y[1])

In [63]:
rdd8 = spark.parallelize([1,2,3,4], 2)

In [64]:
rdd8.aggregate( (0,0) , func1, func2) #초기값(0,0)과 각각 func1과 func2로 지정

(10, 4)

In [ ]:
p1
x=[1,2], zerovalue=[0,0]
x[0] = 0, x[1] = 0
x[0] + y = 0+1 = 1, x[1] + y = 0+1 = 1 >> (1,1)
x[0] + y =1+2 = 3, x[1] + 1 = 1+1 = 2 >> (3,2)

p2
x=[3,4], zerovalue=[0,0]
x[0]=0, x[1]=0
x[0] + y = 0+3 = 3, x[1] + y = 0+1 = 1 >> (3,1)
x[0] + y = 3+4 = 7, x[1] + 1 = 1+1 = 2 >> (7,2)

p1, p2
x = (3,2) y = (7,2)
x[0]+y[0]=3+7=10, x[1]+y[1]=2+2=4 >> (10,4)

In [65]:
rdd8.glom().collect()

[[1, 2], [3, 4]]

## key-value RDD (paired RDD)

groupByKey(),reduceByKey()

그룹핑한 후에 특정 트랜스포메이션을 수행, 키가 필요함. 

In [67]:
rdd9 = spark.parallelize(
    [
        ('짜장면', 15),
        ('짬뽕', 10),
        ('짜장면', 5),
        ('짬뽕', 20)
    ]
)

In [68]:
rdd9_g = rdd9.groupByKey()
rdd9_g

PythonRDD[101] at RDD at PythonRDD.scala:53

In [70]:
rdd9_g.mapValues(len).collect() #개수로 받기

[('짜장면', 2), ('짬뽕', 2)]

In [71]:
rdd9_g.mapValues(list).collect() #리스트로 각 항목을 모두 받기

[('짜장면', [15, 5]), ('짬뽕', [10, 20])]

In [73]:
rdd9_g.mapValues(lambda x: sum(x) / len(x)).collect() #람다 함수로 평균 구하기

[('짜장면', 10.0), ('짬뽕', 15.0)]

In [74]:
rdd9.reduceByKey(add).collect() #대게 reduceBy가 groupBy보다 빠르다. 

[('짜장면', 20), ('짬뽕', 30)]

In [78]:
# countByKey() : 각 키별로 요소들의 개수를 센다. 이때 action함수로 출력
rdd9.countByKey()

defaultdict(int, {'짜장면': 2, '짬뽕': 2})

In [80]:
rdd9.keys().collect()

['짜장면', '짬뽕', '짜장면', '짬뽕']

In [ ]:
# join -outer join, inner join

inner-join : 두개의 집합에서 함께 존재하는 요소(=교집합)의 집합 (default)
outer-join : 한쪽에 있고 다른쪽에 없을때 - left, right


In [82]:
join_rdd1 = spark.parallelize(
    [
        ('a',10),
        ('b',20),
        ('c',30)
    ]
)

join_rdd2 = spark.parallelize(
    [
        ('a',20),
        ('b',20),
        ('b',10)
    ]
)

join_rdd1.join(join_rdd2).collect()

[('b', (20, 20)), ('b', (20, 10)), ('a', (10, 20))]

In [84]:
join_rdd1.leftOuterJoin(join_rdd2).collect()

[('b', (20, 20)), ('b', (20, 10)), ('c', (30, None)), ('a', (10, 20))]

In [85]:
join_rdd1.rightOuterJoin(join_rdd2).collect()

[('b', (20, 20)), ('b', (20, 10)), ('a', (10, 20))]